In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession.builder.master("local")
             .appName("Bronze to Silver")
             .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.11.375")
             .config("spark.hadoop.fs.s3a.access.key", "minio_access_key")
             .config("spark.hadoop.fs.s3a.secret.key", "minio_secret_key")
             .config("spark.hadoop.fs.s3a.path.style.access", "true")
             .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
             .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
             .getOrCreate())


24/05/31 15:05:29 WARN Utils: Your hostname, datdt resolves to a loopback address: 127.0.1.1; using 192.168.2.12 instead (on interface wlp0s20f3)
24/05/31 15:05:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/datdt/anaconda3/envs/de/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/datdt/.ivy2/cache
The jars for the packages stored in: /home/datdt/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e2195a6c-7b21-4eb9-be77-7538feeaa8c3;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 197ms :: artifacts dl 7ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 by [com.amazonaws#aws-java-sdk-bundle;1.11.901] in [default]
	------------------------------------------------------------------

24/05/31 15:05:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.parquet(f"s3a://bronze-data/raw_data")
df.show()

24/05/31 15:05:32 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
+---+-------------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+
|age|          job| marital|education|default|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|
+---+-------------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+
| 19|self-employed|divorced|  unknown|    yes|  -2820|    yes| yes| cellular| 31|  dec|    1933|       4|   15|       3| failure|
| 79| entrepreneur|  single|  primary|    yes|  -5277|    yes| yes|  unknown| 23|  aug|    1919|       8|    3|       7|   other|
| 71| entrepreneur| married|secondary|    yes|  -6381|    yes|  no|  unknown| 10|  sep|     332|       9|   10|       7| failure|
| 78|      student|  single|secondary|    yes|   9437|    yes|  no|telephone|

In [4]:
from pyspark.sql.functions import when

df = df.withColumn('default', when(df['default'] == 'yes', 1).otherwise(0))
df.show()

+---+-------------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+
|age|          job| marital|education|default|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|
+---+-------------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+
| 19|self-employed|divorced|  unknown|      1|  -2820|    yes| yes| cellular| 31|  dec|    1933|       4|   15|       3| failure|
| 79| entrepreneur|  single|  primary|      1|  -5277|    yes| yes|  unknown| 23|  aug|    1919|       8|    3|       7|   other|
| 71| entrepreneur| married|secondary|      1|  -6381|    yes|  no|  unknown| 10|  sep|     332|       9|   10|       7| failure|
| 78|      student|  single|secondary|      1|   9437|    yes|  no|telephone|  9|  mar|    2194|       5|    3|      10| unknown|
| 52|       admin.|  single| tertiary|      1|   8987|    yes| yes|  unknown| 11|  oct|   